Regularization
=============

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']

  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's demonstrate an extreme case of overfitting. Restrict the training set to just a few batches and make a neural network model as before.

In [6]:
seed = 22
batch_size = 128
size_of_hidden = 1024

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, size_of_hidden], seed=seed))
  biases_1 = tf.Variable(tf.zeros([size_of_hidden]))
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  output_1 = tf.nn.relu(logits_1)

  weights_2 = tf.Variable(tf.truncated_normal([size_of_hidden, num_labels], seed=seed))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits_2 = tf.matmul(output_1, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1) + biases_1),
    weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset,weights_1) + biases_1),
    weights_2) + biases_2)

Only use the 1/500 training dataset (= 200 images).

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0]/500 - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 324.513062
Minibatch accuracy: 18.8%
Validation accuracy: 33.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Test accuracy: 77.3%


Introduce L2 regularization method for neural network model. Add L2 penalty on the norm of the weights to the loss function.

In [11]:
seed = 22
batch_size = 128
size_of_hidden = 1024
l2_lambda = 0.01

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, size_of_hidden], seed=seed))
  biases_1 = tf.Variable(tf.zeros([size_of_hidden]))
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  output_1 = tf.nn.relu(logits_1)

  weights_2 = tf.Variable(tf.truncated_normal([size_of_hidden, num_labels], seed=seed))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits_2 = tf.matmul(output_1, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels)) \
    + l2_lambda * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1) + biases_1),
    weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset,weights_1) + biases_1),
    weights_2) + biases_2)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0]/500 - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3475.580078
Minibatch accuracy: 18.8%
Validation accuracy: 33.7%
Minibatch loss at step 500: 21.009052
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 1000: 0.345960
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 1500: 0.189248
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 2000: 0.178698
Minibatch accuracy: 100.0%
Validation accuracy: 73.4%
Minibatch loss at step 2500: 0.176944
Minibatch accuracy: 100.0%
Validation accuracy: 73.4%
Minibatch loss at step 3000: 0.173916
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Test accuracy: 79.6%


Introduce Dropout on the hidden layer of the neural network. Remember that Dropout should only be introduced during training, not evaluation.

In [13]:
seed = 22
batch_size = 128
size_of_hidden = 1024
l2_lambda = 0
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # First Layer
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, size_of_hidden], seed=seed))
  biases_1 = tf.Variable(tf.zeros([size_of_hidden]))
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  output_1 = tf.nn.relu(logits_1)

  # Dropout
  dropped_output_1 = tf.nn.dropout(x=output_1,
                                   keep_prob=keep_prob, seed=seed)

  # Second Layer
  weights_2 = tf.Variable(tf.truncated_normal([size_of_hidden, num_labels], seed=seed))
  biases_2 = tf.Variable(tf.zeros([num_labels])) 
  logits_2 = tf.matmul(dropped_output_1, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels)) \
    + l2_lambda * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),
    weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),
    weights_2) + biases_2)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0]/500 - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 434.444031
Minibatch accuracy: 13.3%
Validation accuracy: 41.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.1%
Minibatch loss at step 1500: 0.022293
Minibatch accuracy: 99.2%
Validation accuracy: 72.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Test accuracy: 79.7%


Learning Rate
=============

Let's add one more layer and use learning rate decay (exponential).

In [9]:
seed = 22
batch_size = 128
size_of_hidden = (1024, 500)
l2_lambda = 1e-05
keep_prob = 1.0

initial_learning_rate = 0.4
decay_steps = 1000
decay_rate = 0.65

first_stddev = np.sqrt(2.0/(image_size * image_size))
second_stddev = np.sqrt(2.0/size_of_hidden[0])
third_stddev = np.sqrt(2.0/size_of_hidden[1])

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # First Layer
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, size_of_hidden[0]], seed=seed,
                                              stddev=first_stddev))
  biases_1 = tf.Variable(tf.zeros([size_of_hidden[0]]))
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  output_1 = tf.nn.relu(logits_1)

  # First Dropout
  dropped_output_1 = tf.nn.dropout(x=output_1, keep_prob=keep_prob, seed=seed)

  # Second Layer
  weights_2 = tf.Variable(tf.truncated_normal([size_of_hidden[0], size_of_hidden[1]], seed=seed,
                                              stddev=second_stddev))
  biases_2 = tf.Variable(tf.zeros([size_of_hidden[1]])) 
  logits_2 = tf.matmul(dropped_output_1, weights_2) + biases_2
  output_2 = tf.nn.relu(logits_2)

  # Second Dropout
  dropped_output_2 = tf.nn.dropout(x=output_2, keep_prob=keep_prob, seed=seed)

  # Third Layer
  weights_3 = tf.Variable(tf.truncated_normal([size_of_hidden[1], num_labels], seed=seed,
                                              stddev=third_stddev))
  biases_3 = tf.Variable(tf.zeros([num_labels]))  
  logits_3 = tf.matmul(dropped_output_2, weights_3) + biases_3

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels)) \
    + l2_lambda * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3))
  
  # Optimizer
  learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate,
                                             global_step=global_step,
                                             decay_steps=decay_steps,
                                             decay_rate=decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits_3)
    
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_layer_2 = tf.nn.relu(tf.matmul(valid_layer_1, weights_2) + biases_2)
  valid_layer_3 = tf.matmul(valid_layer_2, weights_3) + biases_3
  valid_prediction = tf.nn.softmax(valid_layer_3)
    
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_layer_2 = tf.nn.relu(tf.matmul(test_layer_1, weights_2) + biases_2)
  test_layer_3 = tf.matmul(test_layer_2, weights_3) + biases_3
  test_prediction = tf.nn.softmax(test_layer_3)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.374016
Minibatch accuracy: 12.5%
Validation accuracy: 43.7%
Minibatch loss at step 500: 0.523942
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.194567
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 1500: 0.293029
Minibatch accuracy: 93.0%
Validation accuracy: 88.4%
Minibatch loss at step 2000: 0.449229
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 2500: 0.188007
Minibatch accuracy: 96.9%
Validation accuracy: 89.2%
Minibatch loss at step 3000: 0.200096
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Test accuracy: 95.0%
